# Spatial Training

In [1]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from TwoStreamNetwork.spatial.trainModel import get_model, freeze_all_but_top, freeze_all_but_mid_and_top
from TwoStreamNetwork.spatial.trainData import DataSet, get_generators
import time
import os.path
from os import makedirs
from DataSetPathCall import UCF101_PathCall

ucf101 = UCF101_PathCall()

print(ucf101.getDataListPath())

Using TensorFlow backend.


/home/s-sato/dataset/UCF101/data_file.csv


In [2]:
import trainSetting 

trainSetting.GPU_Limit(0.9)
#trainSetting.GPU_LimitAllow()

In [3]:
def train_model(model, nb_epoch, generators, callbacks=[]):
    train_generator, validation_generator = generators
    model.fit_generator(
        train_generator,
        steps_per_epoch=100,
        validation_data=validation_generator,
        validation_steps=10,
        epochs=nb_epoch,
        callbacks=callbacks)
    return model

In [4]:
saved_weights = None
class_limit = None  # int, can be 1-101 or None
num_of_snip = 1 # number of chunks used for each video
image_shape=(224, 224)
load_to_memory = False  # pre-load the sequencea in,o memory
batch_size = 32
nb_epoch = 100
name_str = None

In [5]:
# Get local time.
time_str = time.strftime("%y%m%d%H%M", time.localtime())

if name_str == None:
    name_str = time_str

# Callbacks: Save the model.
directory1 = os.path.join('out', 'checkpoints', name_str)

print(directory1)

if not os.path.exists(directory1):
    os.makedirs(directory1)

checkpointer = ModelCheckpoint(
    filepath=os.path.join(directory1, '{epoch:03d}-{val_loss:.3f}.hdf5'),
    verbose=1,
    save_best_only=True)

# Callbacks: TensorBoard
directory2 = os.path.join('out', 'TB', name_str)
if not os.path.exists(directory2):
    os.makedirs(directory2)
tb = TensorBoard(log_dir=os.path.join(directory2))

# Callbacks: Early stoper
early_stopper = EarlyStopping(monitor='loss', patience=100)

# Callbacks: Save results.
directory3 = os.path.join('out', 'logs', name_str)
if not os.path.exists(directory3):
    os.makedirs(directory3)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join(directory3, 'training-' + \
    str(timestamp) + '.log'))

print("class_limit = ", class_limit)

if image_shape is None:
    data = DataSet(
            class_limit=class_limit,
            data_path=ucf101.getDataListPath()
            )
else:
    data = DataSet(
            image_shape=image_shape,
            class_limit=class_limit,
            data_path=ucf101.getDataListPath()
            )

# Get generators.
generators = get_generators(data=data, image_shape=image_shape, batch_size=batch_size
                            ,trainDir=ucf101.getTrainDir(), validDir=ucf101.getTestDir())

# Get the model.
model = get_model(data=data)


out/checkpoints/1907121742
class_limit =  None
Found 1788425 images belonging to 101 classes.
Found 697865 images belonging to 101 classes.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
if saved_weights is None:
    print("Loading network from ImageNet weights.")
    print("Get and train the top layers...")
    model = freeze_all_but_top(model)
    model = train_model(model, 10, generators)
else:
    print("Loading saved model: %s." % saved_weights)
    model.load_weights(saved_weights)

Loading network from ImageNet weights.
Get and train the top layers...
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
100/100 [==============================] - 875s 9s/step - loss: 4.2326 - acc: 0.1209 - val_loss: 3.2978 - val_acc: 0.2875
Epoch 2/10
100/100 [==============================] - 397s 4s/step - loss: 3.1993 - acc: 0.2794 - val_loss: 3.4245 - val_acc: 0.2781
Epoch 3/10
  2/100 [..............................] - ETA: 5:40 - loss: 2.6734 - acc: 0.4062

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.177772). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 291s 3s/step - loss: 2.6215 - acc: 0.3706 - val_loss: 3.1412 - val_acc: 0.3563
Epoch 4/10
  1/100 [..............................] - ETA: 3:53 - loss: 2.8229 - acc: 0.3125

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.106190). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 287s 3s/step - loss: 2.3843 - acc: 0.4234 - val_loss: 2.8215 - val_acc: 0.3500
Epoch 5/10
  1/100 [..............................] - ETA: 5:49 - loss: 2.1874 - acc: 0.4688

...

  3/100 [..............................] - ETA: 6:35 - loss: 1.6905 - acc: 0.5417

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.103814). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 297s 3s/step - loss: 1.8667 - acc: 0.5153 - val_loss: 3.0089 - val_acc: 0.3969
Epoch 10/10
 12/100 [==>...........................] - ETA: 4:58 - loss: 1.9074 - acc: 0.5052

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.101090). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 309s 3s/step - loss: 1.7400 - acc: 0.5459 - val_loss: 2.8605 - val_acc: 0.4125


In [7]:
print("Get and train the mid layers...")
model = freeze_all_but_mid_and_top(model)
model = train_model(model, 10, generators, [tb, early_stopper, csv_logger, checkpointer])

Get and train the mid layers...
Epoch 1/10
100/100 [==============================] - 302s 3s/step - loss: 1.5371 - acc: 0.6022 - top_k_categorical_accuracy: 0.8384 - val_loss: 2.1495 - val_acc: 0.4844 - val_top_k_categorical_accuracy: 0.7625

Epoch 00001: val_loss improved from inf to 2.14948, saving model to out/checkpoints/1907121742/001-2.149.hdf5
Epoch 2/10
100/100 [==============================] - 274s 3s/step - loss: 1.3709 - acc: 0.6341 - top_k_categorical_accuracy: 0.8656 - val_loss: 2.0863 - val_acc: 0.4844 - val_top_k_categorical_accuracy: 0.7469

Epoch 00002: val_loss improved from 2.14948 to 2.08629, saving model to out/checkpoints/1907121742/002-2.086.hdf5
Epoch 3/10
100/100 [==============================] - 297s 3s/step - loss: 1.3382 - acc: 0.6550 - top_k_categorical_accuracy: 0.8659 - val_loss: 2.0743 - val_acc: 0.5000 - val_top_k_categorical_accuracy: 0.7750

Epoch 00003: val_loss improved from 2.08629 to 2.07432, saving model to out/checkpoints/1907121742/003-2.074

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (1.064031). Check your callbacks.
  % delta_t_median)


  2/100 [..............................] - ETA: 5:01 - loss: 1.1522 - acc: 0.7344 - top_k_categorical_accuracy: 0.8594

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.741309). Check your callbacks.
  % delta_t_median)


  3/100 [..............................] - ETA: 5:34 - loss: 1.1474 - acc: 0.6979 - top_k_categorical_accuracy: 0.8854

...

  2/100 [..............................] - ETA: 4:11 - loss: 0.8867 - acc: 0.8125 - top_k_categorical_accuracy: 0.9062

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.222339). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 293s 3s/step - loss: 1.0818 - acc: 0.7150 - top_k_categorical_accuracy: 0.8956 - val_loss: 2.0717 - val_acc: 0.4750 - val_top_k_categorical_accuracy: 0.7875

Epoch 00007: val_loss did not improve from 1.95295
Epoch 8/10
  3/100 [..............................] - ETA: 6:42 - loss: 1.0425 - acc: 0.7083 - top_k_categorical_accuracy: 0.9062

/home/s-sato/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.153537). Check your callbacks.
  % delta_t_median)


100/100 [==============================] - 301s 3s/step - loss: 1.0769 - acc: 0.7212 - top_k_categorical_accuracy: 0.9025 - val_loss: 2.0699 - val_acc: 0.4594 - val_top_k_categorical_accuracy: 0.7625

Epoch 00008: val_loss did not improve from 1.95295
Epoch 9/10
100/100 [==============================] - 290s 3s/step - loss: 1.0817 - acc: 0.7203 - top_k_categorical_accuracy: 0.9031 - val_loss: 1.9033 - val_acc: 0.4750 - val_top_k_categorical_accuracy: 0.7812

Epoch 00009: val_loss improved from 1.95295 to 1.90329, saving model to out/checkpoints/1907121742/009-1.903.hdf5
Epoch 10/10
100/100 [==============================] - 291s 3s/step - loss: 1.0509 - acc: 0.7181 - top_k_categorical_accuracy: 0.9119 - val_loss: 2.0764 - val_acc: 0.4781 - val_top_k_categorical_accuracy: 0.7531

Epoch 00010: val_loss did not improve from 1.90329


# Temporal Training

In [1]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, LearningRateScheduler
from TwoStreamNetwork.temporal.trainModel import ResearchModels
from TwoStreamNetwork.temporal.trainData import DataSet
import time
import os.path
from os import makedirs
from DataSetPathCall import UCF101_PathCall

ucf101 = UCF101_PathCall()

Using TensorFlow backend.


In [2]:
print(ucf101.getDataListPath())
print(ucf101.getOptFlowDir())

/home/s-sato/dataset/UCF101/tvl1_flow/data_list.csv
/home/s-sato/dataset/UCF101/tvl1_flow


In [3]:
def fixed_schedule(epoch):
    initial_lr = 1.e-2
    lr = initial_lr

    if epoch == 1389:
        lr = 0.1 * lr
    if epoch == 1944:
        lr = 0.1 * lr

    return lr

In [5]:
saved_model = None
class_limit = None  # int, can be 1-101 or None
num_of_snip = 1 # number of chunks used for each video
opt_flow_len = 10 # number of optical flow frames used
image_shape=(224, 224)
load_to_memory = False  # pre-load the sequences into memory
batch_size = 64
nb_epoch = 2222
name_str = None

In [11]:
# Get local time.
time_str = time.strftime("%y%m%d%H%M", time.localtime())

if name_str == None:
    name_str = time_str

# Callbacks: Save the model.
directory1 = os.path.join('out', 'checkpoints', name_str)
if not os.path.exists(directory1):
        os.makedirs(directory1)
checkpointer = ModelCheckpoint(
        filepath=os.path.join(directory1,
                '{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

# Callbacks: TensorBoard
directory2 = os.path.join('out', 'TB', name_str)
if not os.path.exists(directory2):
        os.makedirs(directory2)
tb = TensorBoard(log_dir=os.path.join(directory2))

# Callbacks: Early stopper.
early_stopper = EarlyStopping(monitor='loss', patience=100)

# Callbacks: Save results.
directory3 = os.path.join('out', 'logs', name_str)
if not os.path.exists(directory3):
        os.makedirs(directory3)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join(directory3, 'training-' + \
        str(timestamp) + '.log'))

# Learning rate schedule.
lr_schedule = LearningRateScheduler(fixed_schedule, verbose=0)

print("class_limit = ", class_limit)
# Get the data and process it.
if image_shape is None:
    data = DataSet(
            num_of_snip=num_of_snip,
            opt_flow_len=opt_flow_len,
            class_limit=class_limit,
            opt_flow_path=ucf101.getOptFlowDir(),
            data_path=ucf101.getDataListPath()
            )
else:
    data = DataSet(
            num_of_snip=num_of_snip,
            opt_flow_len=opt_flow_len,
            image_shape=image_shape,
            class_limit=class_limit,
            opt_flow_path=ucf101.getOptFlowDir(),
            data_path=ucf101.getDataListPath()
            )

# Get samples per epoch.
# Multiply by 0.7 to attempt to guess how much of data.data is the train set.
steps_per_epoch = (len(data.data_list) * 0.7) // batch_size
print(data.opt_flow_path)
#print(data.data_list)S

class_limit =  None
/home/s-sato/dataset/UCF101/tvl1_flow


In [8]:
#if load_to_memory:
#    # Get data.
#    X, y = data.get_all_stacks_in_memory('train')
#    X_test, y_test = data.get_all_stacks_in_memory('test')
#else:
# Get generators.
generator = data.stack_generator(batch_size, 'train')

val_generator = data.stack_generator(batch_size, 'test', name_str=name_str)


In [7]:
# Get the model.
temporal_cnn = ResearchModels(nb_classes=len(data.classes), num_of_snip=num_of_snip, 
                              opt_flow_len=opt_flow_len, image_shape=image_shape, 
                              saved_model=saved_model)

Number of classes:
100
Loading CNN model for the temporal stream.
Input shape:
(224, 224, 20)
Numer of classes:
100
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 112, 112, 96)      94176     
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 96)      384       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 96)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2

In [ ]:
# Fit!
if load_to_memory:
    # Use standard fit.
    temporal_cnn.model.fit(
            X,
            y,
            batch_size=batch_size,
            validation_data=(X_test, y_test),
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger],
            epochs=nb_epoch)
else:
    # Use fit generator.
    temporal_cnn.model.fit_generator(
            generator=generator,
            steps_per_epoch=steps_per_epoch,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[tb, early_stopper, csv_logger, checkpointer, lr_schedule],
            validation_data=val_generator,
            validation_steps=1,
            max_queue_size=20,
            workers=1,
            use_multiprocessing=False)